# colab 설정

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import os
from pathlib import Path

folder = "colab/kobert_practice" ## 자기 드라이브 경로 입력

base_path = Path("/content/gdrive/My Drive/")
project_path = base_path / folder
os.chdir(project_path)
for x in list(project_path.glob("*")):
    if x.is_dir():
        dir_name = str(x.relative_to(project_path))
        os.rename(dir_name, dir_name.split(" ", 1)[0])
print(f"{os.getcwd()}")

/content/gdrive/My Drive/colab/kobert_practice


# 패키지 설치

In [3]:
!git clone https://github.com/SKTBrain/KoBERT.git

fatal: destination path 'KoBERT' already exists and is not an empty directory.


In [4]:
cd KoBERT

/content/gdrive/My Drive/colab/kobert_practice/KoBERT


In [5]:
!pip install .

Processing /content/gdrive/My Drive/colab/kobert_practice/KoBERT
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12871 sha256=5250bde3eed680d09fd349cf8d562e527d2d7648f05a35c13e93b3df10cef67f
  Stored in directory: /tmp/pip-ephem-wheel-cache-tkrs__6x/wheels/53/e2/21/b2bac12c9a221a4aa86986c340e3543464e875f2055516f278
Successfully built kobert


In [6]:
!pip install -r requirements.txt
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install transformers==2.1.1
!pip install torch==1.3.1

     |████████████████████████████████| 68.7MB 43kB/s 
     |████████████████████████████████| 256kB 44.2MB/s 
     |████████████████████████████████| 1.1MB 33.7MB/s 
     |████████████████████████████████| 3.9MB 42.1MB/s 
     |████████████████████████████████| 757kB 41.4MB/s 
     |████████████████████████████████| 7.4MB 32.7MB/s 
     |████████████████████████████████| 890kB 36.6MB/s 
     |████████████████████████████████| 3.0MB 31.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470058 sha256=cb9173049d8fd8491c9587ba112f15ec8a9ccf1a0d0b7b9d31e91b5e0318db7c
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=ba8cc0d43b8d1b875a068ab1dc535dc3fe57f3efb124af9c0b0d5213b33a0d15
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully

In [7]:
cd ..

/content/gdrive/My Drive/colab/kobert_practice


# 패키지 로드, class 정의

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.pytorch_kobert import get_pytorch_kobert_model
import pandas as pd
import numpy as np
from kobert.utils import get_tokenizer
from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule
import random

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [13]:
train_data = pd.read_csv('data/ChatbotData .csv')

In [14]:
class ChatDataset(Dataset):
  def __init__(self, dataset, q_idx, a_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
    
    self.question = [transform([dataset.loc[i][q_idx]]) for i in range(len(dataset))]
    self.answer = [transform([dataset.loc[i][a_idx]]) for i in range(len(dataset))]

  def __getitem__(self, i):
    return (self.question[i] + (self.answer[i][0],self.answer[i][1]))
  
  def __len__(self):
    return (len(self.answer))

In [15]:
## 하이퍼 파라미터 설정
max_len = 100
batch_size = 32
warmup_ratio = 0.1
num_epochs = 160
num_workers = 0
max_grad_norm = 1
learning_rate =  5e-5
print_every = 150
save_every = 20
embed_dim = 100 
vocab_size = len(vocab.idx_to_token)

In [16]:
train_dataset = ChatDataset(train_data, "Q", "A", tok, max_len, True, False)

In [17]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True)

In [18]:
class Encoder(nn.Module):
  def __init__(self, bert):
    super(Encoder, self).__init__()
    self.bert = bert
    
  def gen_attention_mask(self, token_ids, valid_length):
    ## masked attenion, 패딩에 패널티 부여, 학습 x
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
        attention_mask[i][:v] = 1
    return attention_mask.float()
  
  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    output, hidden = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

    return output, hidden

In [19]:
class Attention(nn.Module):
  def __init__(self,
               enc_hid_dim,
               dec_hid_dim,
               attn_dim):
    super().__init__()

    self.enc_hid_dim = enc_hid_dim
    self.dec_hid_dim = dec_hid_dim
    self.attn_in = enc_hid_dim + dec_hid_dim
    self.attn = nn.Linear(self.attn_in, attn_dim)
  
  def forward(self, decoder_hidden, encoder_outputs):
    src_len = encoder_outputs.shape[1]

    repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

    energy = torch.tanh(self.attn(torch.cat((
        repeated_decoder_hidden,
        encoder_outputs),
        dim = 2)))

    attention = torch.sum(energy, dim=2)

    return F.softmax(attention, dim=1) 


In [20]:
class Decoder(nn.Module):
  def __init__(self,
               enc_hid_dim,
               hidden_dim,
               emb_dim,
               vocab_size,
               attention,
               max_length,
               dropout_rate = 0.1):
    super(Decoder, self).__init__()

    self.hidden_dim = hidden_dim
    self.dropout_rate = dropout_rate
    self.max_length = max_length
    self.vocab_size = vocab_size
    self.emb_dim = emb_dim
    self.enc_hid_dim = enc_hid_dim

    self.attention = attention

    self.embedding = nn.Embedding(self.vocab_size, self.emb_dim)
    self.gru = nn.GRU(self.enc_hid_dim + self.emb_dim, self.hidden_dim, batch_first=True)
    self.out = nn.Linear(self.attention.attn_in, self.vocab_size)
  
  def _weighted_encoder_rep(self,decoder_hidden, encoder_outputs):
    a = self.attention(decoder_hidden, encoder_outputs)

    a = a.unsqueeze(1)

    weighted_encoder_rep = torch.bmm(a, encoder_outputs)

    return weighted_encoder_rep

  def forward(self, input, decoder_hidden, encoder_outputs):
    
    input = input.unsqueeze(1).long()
    embedded = self.embedding(input)
    weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden, encoder_outputs)
 
    input = torch.cat((embedded, weighted_encoder_rep), dim=2)
    
    output, decoder_hidden = self.gru(input, decoder_hidden.unsqueeze(0))

    embedded = embedded.squeeze(1)
    output = output.squeeze(1)
    weighted_encoder_rep = weighted_encoder_rep.squeeze(1)

    output = self.out(torch.cat((output, weighted_encoder_rep), dim = 1))

    return output, decoder_hidden.squeeze(0)


In [21]:
class Seq2Seq(nn.Module):
  def __init__(self, encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

    self.vocab_size = self.decoder.vocab_size

  def forward(self, src, valid_length, segment_ids, trg, teacher_forcing_ratio = 0.5):
    batch_size = src.shape[0]
    max_len = trg.shape[1]

    encoder_outputs, hidden = self.encoder(src, valid_length, segment_ids)

    outputs = torch.zeros(max_len, batch_size, self.vocab_size).to(self.device)

    output = trg[:,0].long()

    for t in range(1, max_len):
      output, hidden = self.decoder(output, hidden, encoder_outputs)
      outputs[t] = output
      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.max(1)[1]
      output = (trg[:,t] if teacher_force else top1)
    
    return outputs.permute(1,2,0)

In [22]:
## 모델 초기화
encoder = Encoder(bertmodel)
attention = Attention(768, 768, 100)
decoder = Decoder(768, 768, embed_dim, vocab_size, attention, max_len)
model = Seq2Seq(encoder, decoder, device).to(device)

In [23]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [24]:
## 손실함수, optimizer 초기화
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
criterion = nn.CrossEntropyLoss().to(device)

In [25]:
## linear scheduler 초기화
t_total = len(train_loader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [26]:
def save_checkpoint(epoch, model, optimizer, scheduler, loss_list, PATH):
  torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss_list': loss_list
            }, PATH)

In [27]:
def translate_example(inputs, input_length, outputs, label, label_length):
  _, topi = outputs.topk(1)
  for x in [1,2,3,4,5]:
    print("Example #"+str(x))
    s = ""
    for y in inputs[x][:input_length[x]]:
      s += vocab.idx_to_token[y.item()]
    print("input :", s)
    s = ""
    for y in topi[x]:
      s += vocab.idx_to_token[y.item()]
      # if y.item() in [1,3]:
      #   break
    print("pred:", s)
    s = ""
    for y in label[x][:label_length[x]]:
      s += vocab.idx_to_token[y.item()]
    print("label:", s)

# 모델 학습, 불러오기, 결과

In [28]:
save_path = '/content/gdrive/My Drive/colab/kobert_practice/model/checkpoint_2_sample.tar'
checkpoint = torch.load(save_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

In [ ]:
loss_list = []

In [ ]:
print('Training..')
## checkpoint 불러와서 학습시킬 경우 range 범위 바꿔주기
for epoch in range(0, num_epochs):

    model.train()

    for i, (token_ids, valid_length, segment_ids, label, label_length) in enumerate(tqdm_notebook(train_loader)):
        optimizer.zero_grad()

        token_ids, valid_length, segment_ids, label = token_ids.long().to(device), valid_length.to(device), segment_ids.long().to(device), label.long().to(device)

        outputs = model(token_ids, valid_length, segment_ids, label)
  
        loss = criterion(outputs, label)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step() 
        scheduler.step()

        ## accuracy 계산
        _, topi = outputs.permute(0,2,1).topk(1)
        topi = topi.squeeze()
        accuracy = (label == topi).float().mean()

        if (i+1) % print_every == 0: ## 상태 출력
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(
                epoch+1, num_epochs, i+1, len(train_loader), loss.item(), accuracy.item() * 100))
            loss_list.append(loss.item())
            translate_example(token_ids, valid_length, outputs.permute(0,2,1), label, label_length)

        
    ## checkpoint 저장
    if (epoch+1) % save_every == 0:
       if not os.path.exists("model"):
         os.makedirs("model")
       save_checkpoint(epoch+1, model, optimizer, scheduler, loss_list, "model/checkpoint_2_"+str(epoch+1)+".tar")
       print("checkpoint saved!")

Training..


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


Epoch [1/160], Step [150/1223], Loss: 8.7176, Accuracy: 14.47%
Example #1
input : [CLS]▁네▁?[SEP]
pred: [UNK]홍보▁주로▁주로[PAD]붓[PAD]▁주로▁주로▁주로▁주로▁적극적으로▁적극적으로▁적극적으로[PAD]▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁삼성전자▁주로▁주로▁주로▁주로▁中▁주로▁주로▁주로▁상환▁주로▁삼성전자▁삼성전자▁삼성전자▁적극적으로▁주로▁주로▁주로▁폭우▁주로▁적극적으로▁삼성전자▁中▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로붓붓▁주로▁주로▁적극적으로▁봐▁봐▁삼성전자붓▁주로▁주로▁주로▁주로▁봐▁봐▁적극적으로붓▁주로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로▁적극적으로
label: [CLS]▁마굿간에▁있던▁남자애들▁중▁한▁명이▁넬리에게▁네가▁내▁말을▁이미▁샀다고▁말했어▁.▁그리고▁그것은▁신랑이▁그▁말을▁부수고▁있는▁두란의▁농장에▁있다고▁.▁그게▁사실이야▁,▁아빠▁?[SEP]
Example #2
input : [CLS]▁어쨌든▁그▁프랑스▁남자는▁누구야▁.[SEP]
pred: [UNK]엠닌괴괴괴기사기사天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天天
label: [CLS]▁오▁,▁그냥▁어떤▁보험사▁아저씨▁.[SEP]
Example #3
input : [CLS]▁긴▁머리▁관리▁어렵다▁.[SEP]
pred: [UNK]▁결과▁소비자▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된▁공개된

In [ ]:
# for i, (a,b,c,d, e) in enumerate(train_loader):
#   break
# model.eval()
# seq = model(a.long().to(device),b.to(device),c.long().to(device))
# translate_example(a, b, seq, d, e)

Example #1
input : [CLS]▁결혼식▁날짜▁잡았대[SEP]
pred: [CLS]▁축하해주세요▁.[SEP]
label: [CLS]▁축하해주세요▁.[SEP]
Example #2
input : [CLS]▁상처가▁빨리▁나았으면좋겠다[SEP]
pred: [CLS]▁금방▁낫고▁새▁살이을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을을
label: [CLS]▁금방▁낫고▁새▁살이▁돋을▁거예요▁.[SEP]
Example #3
input : [CLS]▁화장이▁안▁받네[SEP]
pred: [CLS]▁각질제거▁먼저하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요하세요
label: [CLS]▁각질제거▁먼저▁하세요▁.[SEP]
Example #4
input : [CLS]▁의욕이▁안▁생겨[SEP]
pred: [CLS]▁하고▁싶은▁걸▁찾아보세요▁[SEP]
label: [CLS]▁하고▁싶은▁걸▁찾아보세요▁.[SEP]
Example #5
input : [CLS]▁뭔▁비밀이▁그렇게▁많은지[SEP]
pred: [CLS]▁하나씩▁비밀을▁공유해보세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요세요
label: [CLS]▁하나씩▁비밀을▁공유해보세요▁.[SEP]


In [29]:
def translate(s):
  
  transform = nlp.data.BERTSentenceTransform(tok, max_seq_length=max_len, pad=True, pair=False)
  token, length, segment = transform([s])
  token = torch.from_numpy(token).to(device)
  length = torch.from_numpy(length).to(device)
  segment = torch.from_numpy(segment).to(device)
  model.eval()
  token = token.view(1,-1)
  length = length.view(1, -1)
  segment = segment.view(1, -1)
  label = torch.zeros(1, max_len).to(device)
  label.fill_(2)
  seq = model(token.long(), length, segment.long(), label, teacher_forcing_ratio=0)
  seq = seq.permute(0,2,1)
  _, topi = seq.topk(1)
  answer = ""
  for x in topi[0]:
    answer += vocab.idx_to_token[x.item()]
    if x.item() in [3]:
        break

  return answer

In [49]:
translate("넌 뭐야?")

'[UNK]▁저는▁위로봇입니다▁.[SEP]'